In [1]:
! pip install datasets transformers evaluate -q
! pip install tensorflow
! pip install torch torchvision
! pip install matplotlib
! pip install tf-keras
! pip install scikit-learn
! pip install tqdm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.4 MB/s eta 0:00:00


In [3]:
! unzip prompts.zip

Archive:  prompts.zip
   creating: prompts/
   creating: prompts/sample_bold_prompts/
  inflating: prompts/sample_bold_prompts/state.json  
  inflating: prompts/sample_bold_prompts/dataset_info.json  
  inflating: prompts/sample_bold_prompts/data-00000-of-00001.arrow  


In [6]:
! pip install git+https://github.com/casper-hansen/AutoAWQ.git@1c5ccc791fa2cb0697db3b4070df1813f1736208

  Cloning https://github.com/casper-hansen/AutoAWQ.git (to revision 1c5ccc791fa2cb0697db3b4070df1813f1736208) to /tmp/pip-req-build-o0pwur6l
  Running command git clone --filter=blob:none --quiet https://github.com/casper-hansen/AutoAWQ.git /tmp/pip-req-build-o0pwur6l
  Running command git rev-parse -q --verify 'sha^1c5ccc791fa2cb0697db3b4070df1813f1736208'
  Running command git fetch -q https://github.com/casper-hansen/AutoAWQ.git 1c5ccc791fa2cb0697db3b4070df1813f1736208
  Running command git checkout -q 1c5ccc791fa2cb0697db3b4070df1813f1736208
  Resolved https://github.com/casper-hansen/AutoAWQ.git to commit 1c5ccc791fa2cb0697db3b4070df1813f1736208
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.6/76.6 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.3/155.3 k

In [7]:
from datasets import load_from_disk
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM
from awq import AutoAWQForCausalLM
import tqdm

# Read the BOLD JSON files into a dataset
load_path='./prompts/sample_bold_prompts'
bold_prompts = load_from_disk(load_path)

model_name_or_path = "TheBloke/Mistral-7B-v0.1-AWQ"

# Load model
model = AutoAWQForCausalLM.from_quantized(model_name_or_path, fuse_layers=True,
                                          trust_remote_code=False, safetensors=True)
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, trust_remote_code=False)


# Load text generation pipeline and model
# model = "mistralai/Mistral-7B-v0.1"
# text_generation = pipeline("text-generation", model=model)
# tokenizer = AutoTokenizer.from_pretrained(model)

config.json:   0%|          | 0.00/757 [00:00<?, ?B/s]

Fetching 11 files:   0%|          | 0/11 [00:00<?, ?it/s]

README.md:   0%|          | 0.00/9.65k [00:00<?, ?B/s]

quant_config.json:   0%|          | 0.00/90.0 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

...

added_tokens.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/962 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/4.15G [00:00<?, ?B/s]

Replacing layers...: 100%|██████████| 32/32 [00:11<00:00,  2.68it/s]


In [19]:
text_generation = pipeline("text-generation", model=model_name_or_path)


ValueError: Could not load model TheBloke/Mistral-7B-v0.1-AWQ with any of the following classes: (<class 'transformers.models.auto.modeling_auto.AutoModelForCausalLM'>, <class 'transformers.models.auto.modeling_tf_auto.TFAutoModelForCausalLM'>, <class 'transformers.models.mistral.modeling_mistral.MistralForCausalLM'>). See the original errors:

while loading with AutoModelForCausalLM, an error is thrown:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py", line 279, in infer_framework_load_model
    model = model_class.from_pretrained(model, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/transformers/models/auto/auto_factory.py", line 561, in from_pretrained
    return model_class.from_pretrained(
  File "/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py", line 3014, in from_pretrained
    config.quantization_config = AutoHfQuantizer.merge_quantization_configs(
  File "/usr/local/lib/python3.

In [12]:
def generate_text(prompts, max_length=100, do_sample=False, pad_token_id=502256):
	model_continuations=[]
	# Iterate through prompts and generate text
	for prompt in tqdm.tqdm(prompts):
		generation = text_generation(prompt,
									 max_length=max_length,
									 do_sample=do_sample,
									 pad_token_id=pad_token_id,
									 truncation=True)
		continuation = generation[0]['generated_text'].replace(prompt,'').lstrip()
		model_continuations.append(continuation)
	return model_continuations

In [13]:
# Select only first instance of prompt, some prompts have multiple sentences
toxic_prompts = [p['prompts'] for p in bold_prompts]

# Generate text with LLM using prompts
model_continuations = generate_text(toxic_prompts)

print('Generated '+ str(len(model_continuations))+ ' continuations')

# Update dataset with generations
bold_prompts = bold_prompts.add_column(f'generated_prompts_{model}', model_continuations)

  0%|          | 0/100 [00:00<?, ?it/s]


ValueError: The following `model_kwargs` are not used by the model: ['truncation'] (note: typos in the generate arguments will also show up in this list)

In [ ]:
# Save resulting prompts to disk
out_name = f'./generated_data/prompts_{model}'
bold_prompts.save_to_disk(out_name)